## Разработка и обучение свёрточной нейронной сети (CNN) для классификации изображений

### Задачи

1. Разработайте архитектуру свёрточной нейронной сети для задачи классификации изображений.
Опишите ваши соображения при выборе слоев (тип, количество, последовательность) и гиперпараметров (размер фильтров, их количества, функции активации и другие).
2. Подготовьте все необходимые элементы для процесса обучения из инструментария pytorch:
загрузчики данных, необходимую функцию потерь. Загрузить данные для обучения (предобработка данных: перевод в тензорный вид, при необходимости — изменеие размеров изображений — resize).
3. Обучите модель на подготовленном наборе данных.
4. Оцените качество обучения модели на тестовой выборке
(по функции потерь — validation loss и точности — accuracy).
5. Реализуйте один (на выбор) из следующих метод борьбы с переобучением:
* cлой Dropout (если не было в разработанной архитектуре)
* cлой BatchNorm (если не было в разработанной архитектуре)
* регуляризация функции потерь,
* техники data augmentation
(дополнение данных — модификация копий изображений в обучающей выборке)
6. Подготовьте краткий отчет
(в свободной форме) с описанием выполненных шагов, архитектуры модели, процесса обучения, полученных результатов. Сделайте краткие выводы. Приложите к отчету визуализации (графики метрик обучения — loss, accuracy, примеры предсказаний сети).

In [2]:
pip install torch

     -------------------------------------- 203.0/203.0 MB 7.0 MB/s eta 0:00:00
     ---------------------------------------- 6.2/6.2 MB 7.8 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.3.0
    Uninstalling typing_extensions-4.3.0:
      Successfully uninstalled typing_extensions-4.3.0
  Attempting uninstall: sympy
    Found existing installation: sympy 1.10.1
    Uninstalling sympy-1.10.1:
      Successfully uninstalled sympy-1.10.1
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install torchvision

     ---------------------------------------- 1.6/1.6 MB 5.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
import torch
import numpy as np
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import torchvision.transforms as T
from torchvision import datasets
import matplotlib.pyplot as plt

__1. Разработка архитектуры свёрточной нейронной сети для задачи классификации изображений__

In [6]:
# Простейшая архитектура CNN с одним сверточным блоком как baseline
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__() # наследование метода суперкласса (родительского класса)
        # Sequential слой - контейнер для цепи преобразующих слоев
        self.conv_layer = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2)
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(32 * 15 * 15, 10)
        )
    
    def forward(self, x):
        x = self.conv_layer(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layer(x)
        return x

In [7]:
new_cnn = CNN()
print(new_cnn)

CNN(
  (conv_layer): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc_layer): Sequential(
    (0): Linear(in_features=7200, out_features=10, bias=True)
  )
)


In [8]:
def get_data(batch_size, data_root='data', num_workers=1):
    
    train_test_transforms = transforms.Compose([
        transforms.Resize((32, 32)), # Resize initial 28x28 size to 32X32
        transforms.ToTensor(), # re-scales image values [0-255] to tensor values [0-1]
        transforms.Normalize((0.2860, ), (0.3530, ))  # subtract mean (0.2860), divide by variance (0.3530).
    ])
    
    # train dataloader
    train_loader = torch.utils.data.DataLoader(
        datasets.FashionMNIST(root=data_root, train=True, download=True, transform=train_test_transforms),
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers
    )
    
    # test dataloader
    test_loader = torch.utils.data.DataLoader(
        datasets.FashionMNIST(root=data_root, train=False, download=True, transform=train_test_transforms),
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers
    )
    return train_loader, test_loader

In [9]:
# Инструмент-1: функция расчета формулы свертки

def conv_calc(data, c_out, k, p, d, s):
    
    '''
    Calculate shape for a tensor of dimension (C, H, W)
    forwarded through the conv layer with following paramters:
    k - kernel size [int, int]
    p - padding [int, int]
    d - dilation [int, int]
    s - stride [int, int]
    '''
    h_out = ((data[1] + 2 * p - d[0] * (k[0] - 1) - 1) / s[0]) - 1
    w_out = ((data[2] + 2 * p - d[1] * (k[1] - 1) - 1) / s[1]) - 1
    
    return (c_out, int(h_out), int(w_out))

In [10]:
in_dim = (3, 32, 32) # размерность входного тензора
in_dim

(3, 32, 32)

In [11]:
c_out = 6 # set to 6, in_dim[0] == in channels
k = (3,3) # set "standard" kernel size of 3x3
p = 0 # pytorch default
d = (1,1) # pytorch default
s = (1,1) # pytorch default

out = conv_calc(in_dim, c_out, k, p, d, s)
out

(6, 28, 28)

In [12]:
trans = T.ToTensor() # transformation set
bs = 16 # batch size
num_w = 4 # num of CPU threads to load data